In [1]:
import BoardGame
import Player
from joblib import Parallel, delayed
import multiprocessing
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from keras.models import load_model

Using TensorFlow backend.
/home/caio/Documents/2048/2048env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/caio/Documents/2048/2048env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/caio/Documents/2048/2048env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/caio/Documents/20

In [2]:
def opa(i):
    d.scores[i] = d.players[i].play_game(BoardGame.board())

In [3]:
class Darwin(object):
    
    def __init__(self, n=10):
        
        self.players = {i: Player.Player() for i in range(n)}
        self.scores = {i: 0 for i in range(n)}
        self.n = n
        self.history = []
        
    def trainning(self, epochs=2):
        
        for epoch in tqdm(range(epochs)):
            for n in range(10):
                for i in range(self.n):
                    self.scores[i] = self.players[i].play_game(BoardGame.board())
            
            best = min(self.scores, key=self.scores.get)
#             print('Melhor jogador: '+str(best))
#             print('Score:'+str(self.scores[best]))
            self.history.append(self.scores[best])
            self.remove_worst()
            self.reset_scores()
            
            
    def remove_worst(self):
        
        for i in range(int(self.n * 0.9)):
            worst = min(self.scores, key=self.scores.get)
            self.players[worst].evolve()
            self.scores[worst] = np.inf
    
    def reset_scores(self):
        
        self.scores = {i: 0 for i in range(self.n)}
    
    def plot_history(self):
        
        plt.plot(self.history[-100:], color='crimson')
        plt.title('Melhor score por época')
        plt.xlabel('Época')
        plt.ylabel('Score')
        plt.grid(True)
        plt.savefig('history_scores.png')
        
    def get_best(self):
        
        self.players[max(self.scores, key=self.scores.get)].save_model()
        return self.players[max(self.scores, key=self.scores.get)]
    
    def recall_players(self, n):
        
        self.__init__(n)
        for i in range(self.n):
            self.players[i].model = load_model('best_player.h5')
        
        